In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import joblib
import csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

In [15]:
def shadow_removalDialate(img):
    # img2 = cv2.imread(f'../Dataset_0-5/men/{num}/4_men (49).JPG')
    img2 = cv2.resize(img, (256, 256))
    
    img_hls = cv2.cvtColor(img2, cv2.COLOR_BGR2HLS)

    # define lower and upper bounds for blue color in HLS format
    lower_blue = np.array([0, 0, 60])
    upper_blue = np.array([20, 255, 255])
    # create a mask for blue color in HLS format
    mask = cv2.inRange(img_hls, lower_blue, upper_blue)
    # daialation
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 8)
    # apply the mask to the original image
    result = cv2.bitwise_and(img2, img2, mask=mask)
    return result

In [16]:
def preprocessing(img,img2):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to enhance contrast and remove lighting effects
    equalized = cv2.equalizeHist(gray)
    # io.imshow(equalized)
    # # Find the contours of the binary image
    contours, hierarchy = cv2.findContours(equalized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)

    # # Draw the largest contour on the original image
    cv2.drawContours(img2, [max_contour], 0, (0, 255, 0), 2)
    # io.imshow(img2)
    # # Show the image with the largest contour drawn
    # # Create a bounding box around the hand
    x, y, w, h = cv2.boundingRect(max_contour)

    # # Crop the image to the bounding box around the hand
    hand = equalized[y:y+h, x:x+w]
    # added 
    Noise_Reduction = cv2.medianBlur(hand, 5)
    img3 = cv2.resize(Noise_Reduction, (256, 256))
    # io.imshow(img3)

    return img3

In [17]:



arr = ["0","1","2","3","4","5"]
data=[]
hog_features = []
lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))

            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            hog = Hog_descriptor(preprocessed_image, cell_size=4, bin_size=9)
            vector, image = hog.extract()
            vec = np.array(vector)
            hog_features.append(vec.flatten())
            data.append(label)
        except:
            pass


In [18]:

arr = ["0","1","2","3","4","5"]
for label in arr:
    dirList = glob.glob("../Dataset_0-5/Women/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            hog = Hog_descriptor(preprocessed_image, cell_size=4, bin_size=9)
            vector, image = hog.extract()
            vec = np.array(vector)
            hog_features.append(vec.flatten())
            data.append(label)

        except:
            pass


In [21]:
train_x,test_x,train_y,test_y = train_test_split(hog_features,data, test_size = 0.2)
classifier=SVC(kernel="linear", random_state=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/models3")

['model/models3']

In [22]:
prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.7671232876712328
Precision=  0.7671103413948781
Recall=  0.7671232876712328
F1 Score=  0.7661056170293966
Confusion Matrix=  [[50  0  0  1  1  1]
 [ 0 59  5  1  0  2]
 [ 5  5 29  9  3  1]
 [ 1  2  6 37  8  3]
 [ 1  1  3 10 46  2]
 [ 2  1  4  4  3 59]]
Classification Report=                precision    recall  f1-score   support

           0       0.85      0.94      0.89        53
           1       0.87      0.88      0.87        67
           2       0.62      0.56      0.59        52
           3       0.60      0.65      0.62        57
           4       0.75      0.73      0.74        63
           5       0.87      0.81      0.84        73

    accuracy                           0.77       365
   macro avg       0.76      0.76      0.76       365
weighted avg       0.77      0.77      0.77       365



In [ ]:
with open('data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(hog_features)):
        # Write one row at a time, with the label in the first column
        row = [data[i]] + list(hog_features[i])
        writer.writerow(row)